# Convertendo Áudio para Texto

No processo do desenvolvimento de algumas Pesquisas, sejam elas de mercado ou até acadêmicas, é feita uma série de entrevistas com pessoas que possuam o conhecimento necessário que satisfaz o pesquisador, sendo elas consumidores, em casos de pesquisa de mercado, até especialistas, alunos, professores e pesquisadores, para coleta das informações que possam colaborar na conclusão de melhores resultados.

Nas entrevistas, é anotado o que o entrevistado falou, e muitas vezes, para facilitar o armazenamento dessas informações, fazemos gravações de áudio, ou até de vídeo, possibilitando transformação das informações em dados, viabilizando até a "quantificação" dos dados para a pesquisa. Porém, durante a análise do que foi coletato, não é tão simples encontrar certas palavras, falas ou assuntos ditos, no áudio gravado, caso a entrevista tenha uma longa duração. Então, uma solução que ajuda na análise é a Conversão/Transcrição do áudio para textos. É mais simples encontrar em qual momento tal assunto ou palavra foi dita em um arquivo de texto, ao inves de procurá-la no arquivo de áudio, utilizando a navegação de um aplicativo de Player de Áudio.

Para a conversão, foi necessário o uso de uma biblioteca open source de "Speech Recognition" (Reconhecimento de Fala). 

### O que é a  biblioteca "Speech Recognition"?

É uma biblioteca que utiliza bases de inteligência artificial para reconhecimento de falas, em vários idiomas, que é muito utilizada em ferramentas e aplicativos como Google Tradutor e até no YouTube, com sua função de legendar os vídeos em tempo real.

Como tudo na Técnologia da Informação, há muito para a biblioteca chegar a 100% de exatidão na sua funcionalidade mas, com o que já temos dela hoje, funciona muito bem em várias aplicações, e por ser open source, a comunidade de Programadores e Cientistas de Dados está sempre contribuindo para a melhorá-la. A biblioteca está disponível para uso em muitas linguagens de programação, e para esses testes foi utilizada na linguagem Python.

Além do reconhecimento de falas em arquivos de áudio, é possível desenvolver scripts que reconheçam as falas e convertam para texto em tempo real, como no exemplo à seguir:

In [ ]:
import speech_recognition as sr
# Obtem o áudio do microfone
r = sr.Recognizer()
with sr.Microphone() as source:
    print("Say something!")
    audio = r.listen(source)
    print("Time's up! Thanks!")
    
try:
    print("Text: "+ r.recognize_google(audio))

except sr.UnknownValueError:
    pass;

### Teste de conversão de Áudio de Longa Duração

Nos teste iniciais, os resultados não foram satisfatorios na leitura do arquivo de áudio bruto pelo script. O primeiro script retornava apenas os segundos iniciais do áudio. Por isso, foi preciso dividir o áudio bruto em vários outros áudios com duração de 30 segundos cada.

Para isso, foi utilizado outra biblioteca open source chamada "ffmpeg". Ela já é pré-instalada em alguns sistemas Linux e na aplicação "Anaconda3" (gerenciador de ambientes de desenvolvimento de códigos) no Windows.

O comando a seguir recebe o áudio bruto como entrada, recebe o comando de "divisões" de duração de 30 segundos e salva os novos áudios na pasta "parts":

In [ ]:
ffmpeg -i 20160229-111428.wav -f segment -segment_time 30 -c copy parts/out%09d.wav

Para a transcrição de falas, foi desenvolvido o script a seguir:

In [ ]:
import os
import speech_recognition as sr
from tqdm import tqdm

with open("api-key.json") as f:
    GOOGLE_CLOUD_SPEECH_CREDENTIALS = f.read()

r = sr.Recognizer()
files = sorted(os.listdir('parts/'))

all_text = []

for f in tqdm(files):
    name = "parts/" + f
    # Arquivo de Audio Local
    with sr.AudioFile(name) as source:
        audio = r.record(source)
    # Transcrevendo o Arquivo de Áudio
    text = r.recognize_google_cloud(audio, credentials_json=GOOGLE_CLOUD_SPEECH_CREDENTIALS, language="pt-BR")
    all_text.append(text)

transcript = ""
for i, t in enumerate(all_text):
    total_seconds = i * 30
    m, s = divmod(total_seconds, 60)
    h, m = divmod(m, 60)

    # Formatar tempo em h:m:s - 30 segundos de texto
    transcript = transcript + "{:0>2d}:{:0>2d}:{:0>2d} {}\n".format(h, m, s, t)

print(transcript)

with open("transcripted_text.txt", "w") as f:
    f.write(transcript)

Após a criação dos áudios, o script de transcrição de áudio para texto é executado. Este script faz a leitura de cada áudio da pasta "parts" em ordem númerica. Em cada leitura, é feito o reconhecimento da fala atravéz da integração da biblioteca "Speech Recognition" com a base de inteligencia artificial do Google, na qual o retorno da leitura é em texto, os textos são armazenados de forma cronológica para que no final todos eles sejam gravados em um arquivo de texto no formato .txt.

Aproximadamente 80% das falas foram transcritas, e a precisão no reconhecimento é influenciada por fatores como volume de áudio, e tudo isso indica que melhores conversões/resultados podem ser obtidos no madurecimento do trabalho, tanto de gravação de áudio quanto no desenvolvimento do script.